In [1]:
import torch
import numpy as np
import pandas as pd
import os
import datetime
from datetime import date, timedelta
from graph import Graph
from utils.data_utils import data_to_numpy

In [2]:
unlabelled = pd.read_csv('data/la_train_grid_with_weather.csv')
training_data = pd.read_csv('data/LA_DATA_2018_02_to_2018_06.csv')


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# node_cols = 'visibility', 'precipitation_depth']
node_cols = ['wind_x', 'wind_y', 'temperature', 'pressure', 'ceiling', 'dew', 'precipitation_duration' , 'mean_aod','min_aod','max_aod', 'pm25']                                                 
edge_cols = ['wind_x', 'wind_y']
date_range = sorted(training_data['DATE'].unique(), key=lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

def assign_id(weather_data):
    ids = np.zeros(len(weather_data['Latitudes']))
    node_pos = list(weather_data.groupby(['Latitudes', 'Longitudes']).groups)
    for i, node in enumerate(node_pos):
        indices_lats = weather_data['Latitudes'].values == node_pos[i][0]
        indices_longs = weather_data['Longitudes'].values == node_pos[i][1]
        indices = indices_lats & indices_longs
        ids[np.argwhere(indices)] = int(i+1)
    weather_data['STATION'] = ids
    return weather_data

def get_unlabelled_subset(unlabeled_df, dates, node_cols, edge_cols, meta_cols):
    cols = list(set(node_cols + edge_cols + meta_cols))
    unlabeled_df = unlabeled_df[cols]
    unlabeled_df = unlabeled_df[unlabeled_df['day'].isin(dates)]
    unlabeled_df = assign_id(unlabeled_df)
    unlabeled_df['pm25'] = [np.nan for _ in range(len(unlabeled_df))]
    return unlabeled_df






# graph_node_features, graph_edge_features, graph_labels

In [5]:
node_cols = ['wind_x', 'wind_y', 'temperature', 'pressure', 'ceiling', 'dew', 'precipitation_duration' , 'mean_aod','min_aod','max_aod']                                                 
edge_cols = ['wind_x', 'wind_y']
meta_cols = ['day', 'Latitudes', 'Longitudes']
date_range = sorted(training_data['DATE'].unique(), key=lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))

unlabeled_df = get_unlabelled_subset(unlabelled, date_range,  node_cols, edge_cols, meta_cols)

In [6]:
dataset = unlabeled_df.append(training_data, ignore_index=True)

In [7]:
graph_node_features, graph_edge_features, graph_labels = data_to_numpy(dataset, edge_cols, node_cols, pseudodata = True)

Found Checkpoint, loading


In [144]:
unlabeled_df

,temperature,Longitudes,ceiling,mean_aod,min_aod,Latitudes,precipitation_duration,day,pressure,wind_y,dew,wind_x,max_aod,STATION,pm25
0,NaN,33.664840,NaN,0.151435,0.117,-118.225777,NaN,2018-02-01,10150.29167,NaN,NaN,NaN,0.193,106.0,NaN
2,NaN,33.664840,NaN,0.097443,0.066,-118.225777,NaN,2018-02-03,10185.12500,NaN,NaN,NaN,0.147,106.0,NaN
3,NaN,33.664840,NaN,0.098783,0.068,-118.225777,NaN,2018-02-04,10172.12500,NaN,NaN,NaN,0.138,106.0,NaN
4,NaN,33.664840,NaN,0.140263,0.103,-118.225777,NaN,2018-02-05,10154.95833,NaN,NaN,NaN,0.435,106.0,NaN
5,NaN,33.664840,NaN,0.180686,0.144,-118.225777,NaN,2018-02-06,10150.91667,NaN,NaN,NaN,0.224,106.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
468904,23.3675,34.186204,22000.00000,0.121316,0.034,-117.237630,NaN,2018-06-03,NaN,-53.536863,86.700000,-82.459021,0.261,450.0,NaN
468905,23.3425,34.186204,19765.64103,0.094231,0.019,-117.237630,NaN,2018-06-04,NaN,-12.773724,97.100000,-10.473038,0.195,450.0,NaN
468906,20.1475,34.186204,16012.67500,0.179752,0.108,-117.237630,NaN,2018-06-05,NaN,-46.484691,122.400000,-11.883327,0.341,450.0,NaN
468907,19.6375,34.186204,12318.05000,0.161026,0.074,-117.237630,NaN,2018-06-06,NaN,-23.033544,117.050000,-28.418109,0.284,450.0,NaN
